In [3]:
# Install necessary libraries if you haven't already
# !pip install pypdf
# !pip install langchain
# !pip install langchain-google-genai
# !pip install chromadb
# !pip install -U langchain-community
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

import os
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

print("Libraries imported successfully!")







Libraries imported successfully!


In [2]:
#!pip install langchain_google_genai

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------------------------------ --------- 1.0/1.4 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.2 MB/s eta 0:00:00

  Attempting uninstall: protobuf

    Found existing installation: protobuf 5.29.3

   ---------- ----------------------------- 1/4 [protobuf]
    Uninstalling protobuf-5.29.3:
   ---------- ----------------------------- 1/4 [protobuf]
      Successfully uninstalled protobuf-5.29.3
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ---------- ----------------------------- 1/4 [protobuf]
   ----------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogen-core 0.6.4 requires protobuf~=5.29.3, but you have protobuf 6.32.0 which is incompatible.


In [5]:
# --- 1. Set up the API Key ---
# For security, use getpass to hide your API key input.
# Or, you can set it as an environment variable.
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("you google code")
    print("\nAPI key set successfully!")


you google code ········



API key set successfully!


In [9]:
#!pip install pymupdf

   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.7 MB 4.2 MB/s eta 0:00:05
   -- ------------------------------------- 1.3/18.7 MB 4.5 MB/s eta 0:00:04
   --- ------------------------------------ 1.6/18.7 MB 2.7 MB/s eta 0:00:07
   ----- ---------------------------------- 2.4/18.7 MB 3.1 MB/s eta 0:00:06
   ----- ---------------------------------- 2.6/18.7 MB 3.3 MB/s eta 0:00:05
   ----- ---------------------------------- 2.6/18.7 MB 3.3 MB/s eta 0:00:05
   ----- ---------------------------------- 2.6/18.7 MB 3.3 MB/s eta 0:00:05
   ------- -------------------------------- 3.4/18.7 MB 2.2 MB/s eta 0:00:08
   -------- ------------------------------- 4.2/18.7 MB 2.3 MB/s eta 0:00:07
   --------- ------------------------------ 4.5/18.7 MB 2.1 MB/s eta 0:00:07
   ----------- ---------------------------- 5.2/18.7 MB 2.3 MB/s eta 0:00:06
   ------------ --------------------------- 6.0/18.7 MB 2.5 MB/s eta 0:00:06
   ---

In [10]:
# --- 2. Load the Document ---
# Define the path to your PDF file.
# The file 'BELT STOCK.pdf' should be in the same directory as this notebook.
file_path = "BELT STOCK.pdf"
try:
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    print(f"\nLoaded {len(documents)} pages from {file_path}")
except FileNotFoundError:
    print(f"\nError: The file '{file_path}' was not found. Please ensure it is in the same directory.")
    # Exit gracefully if the file is not found
    exit()


Loaded 2 pages from BELT STOCK.pdf


In [11]:
# --- 3. Split Documents into Chunks ---
# We split the documents into smaller, manageable chunks.
# This helps the model focus on relevant information without being overwhelmed.
# The chunk size and overlap can be tuned for better performance.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"Split {len(documents)} pages into {len(docs)} chunks.")


Split 2 pages into 2 chunks.


In [12]:
# --- 4. Initialize Embedding Model and Vector Store ---
# An embedding model converts text into numerical vectors.
# We'll use Google's embedding model for this.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# We'll use Chroma as our vector store to store the document chunks and their embeddings.
# This allows us to quickly search for relevant chunks later.
vectorstore = Chroma.from_documents(docs, embeddings)
print("Vector store created successfully.")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Vector store created successfully.


In [13]:
# --- 5. Create the Retriever ---
# The retriever is a key component of RAG. It's responsible for fetching
# the most relevant document chunks based on a user's query.
retriever = vectorstore.as_retriever()
print("Retriever created.")

Retriever created.


In [14]:
# --- 6. Set up the RAG Chain ---
# First, define the prompt template that will guide the LLM's response.
# The prompt tells the model to use the provided context to answer the question.
prompt = ChatPromptTemplate.from_template("""
Answer the user's question based only on the provided context.
If you do not have enough information in the context, please state that you cannot answer.

Context:
{context}

Question:
{input}
""")

In [17]:
# Initialize the LLM with the gemini-2.5-flash model.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Create a 'stuff' documents chain. This chain takes the retrieved documents
# and 'stuffs' them into the prompt's context.
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the final retrieval chain. This chain combines the retriever and the document chain.
# When a user asks a question, this chain will:
# 1. Retrieve relevant documents using the retriever.
# 2. Pass those documents to the document chain for generation.
retrieval_chain = create_retrieval_chain(retriever, document_chain)
print("RAG chain setup complete.")


RAG chain setup complete.


In [22]:
# --- 7. Test the Agent ---
print("\n--- Testing the RAG Agent with a specific question ---")
question = "What is the total belt stock?"

# Invoke the chain with the question.
response = retrieval_chain.invoke({"input": question})

# Print the final answer from the RAG agent.
print("\nQuestion:", question)
print("\nAnswer:")
print(response["answer"])



--- Testing the RAG Agent with a specific question ---


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Question: What is the total belt stock?

Answer:
TOTAL BELT STOCK - 8961METERS


In [ ]:
# --- 8. Optional: Clean up ---
# You can delete the Chroma collection if you're done.
# vectorstore.delete_collection()
# print("\nVector store collection deleted.")


In [20]:
#--- 7. Test the Agent with multiple questions ---
print("\n--- Testing the RAG Agent with a list of questions ---")

# Define a list of questions you want to ask.
questions = [
    "What is the belt drum details of BELT STRENGTH AND BELT WIDTH EP800/4 (1600MM)?",
    "What is the total stock belt in meters for the ST 2000 (2000 mm) belt?",
    "Give me the belt drum details of ST 1400 (2000mm).",
    "What are the belt drum details for the EP800/4 belt with a 2400mm width?"
]

# Loop through the questions and get the answers.
for question in questions:
    print("-" * 50)
    print(f"Question: {question}")
    response = retrieval_chain.invoke({"input": question})
    print("Answer:")
    print(response["answer"])




--- Testing the RAG Agent with a list of questions ---
--------------------------------------------------
Question: What is the belt drum details of BELT STRENGTH AND BELT WIDTH EP800/4 (1600MM)?


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Answer:
The belt drum details for BELT STRENGTH AND BELT WIDTH EP800/4 (1600MM) is 99meter-1.
--------------------------------------------------
Question: What is the total stock belt in meters for the ST 2000 (2000 mm) belt?


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Answer:
Based on the provided context, the total stock belt for ST 2000 (2000 mm) is 2565 meters.
--------------------------------------------------
Question: Give me the belt drum details of ST 1400 (2000mm).


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Answer:
I cannot answer this question as the provided context does not contain the belt drum details for ST 1400 (2000mm).
--------------------------------------------------
Question: What are the belt drum details for the EP800/4 belt with a 2400mm width?


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Answer:
I cannot answer what the belt drum details are for the EP800/4 belt with a 2400mm width, as this information is not provided in the context.


In [32]:
system_prompt =  """Answer ONLY in tabular format.
Each row must represent one belt entry from the BELT STOCK table.
Columns: Belt Strength and width | Belt stock| drum details (m) | Number of Drums | make | .
Do not add explanations outside the table."""
               


question = "What is the belt drum details of belt strength and widthST2000(2000mm)?"

response = retrieval_chain.invoke({"input": f"{system_prompt}\n\nQuestion: {question}"})

print("\nQuestion:", question)
print("\nAnswer (Table):")
print(response["answer"])


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Question: What is the belt drum details of belt strength and widthST2000(2000mm)?

Answer (Table):
I cannot answer the question as the provided context does not contain the belt drum details for ST 2000 (2000mm).
